# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# A brief history of solver settings

We will run an relatively large example model with a different solver setting to evalute their effect on model convergence. Before starting download and unzip [solver.zip](https://github.com/jdhughes-usgs/GW3099-2024/releases/download/solver_example_data/solver.zip) and unzip the data files to the `data` directory. The model input files in the zip file should be in the `../../data/solver/` directory.

In [ ]:
import pathlib as pl

import flopy
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def read_solver_csv():
    """
    Function to create a pandas dataframs from the solver csv file.
    """
    fpath = ws / "ims.inner.csv"
    return pd.read_csv(fpath)

Set processors to None to run with the IMS solver 1 to run with the PETSc solver.

In [ ]:
processors = None

Set path to raw model data files and where to write the model files.

In [ ]:
data_ws = pl.Path("../../data/solver/")

In [ ]:
name = "solver"
ws = pl.Path(f"temp/{name}")

## Load the existing model files

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=data_ws)

In [ ]:
gwf = sim.get_model()

Plot the model files

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
v = mm.plot_array(gwf.dis.top.array, masked_values=[2.99999989e30])
plt.colorbar(v, shrink=0.75);

## Run the base simulation

In [ ]:
sim.set_sim_path(ws)
sim.write_simulation()

In [ ]:
sim.run_simulation(processors=processors)

### Read the solver inner iteration csv file and plot the `inner_dvmax`

In [ ]:
df_index = ["base"]
df = read_solver_csv()
df

In [ ]:
plt.plot(df.total_inner_iterations, df.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Increase number of outer iterations

Modify `outer_maximum` from 100 to 300

In [ ]:
sim.ims.outer_maximum = 300
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("max_iter=300")
df1 = read_solver_csv()
df1

In [ ]:
plt.plot(df.total_inner_iterations, df.solution_inner_dvmax)
plt.plot(df1.total_inner_iterations, df1.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Add simple under relaxation

Change `outer_maximum` back to 100, set `under_relaxation` to `simple`, write the revised `IMS` file, and run the model 

In [ ]:
sim.ims.outer_maximum = 100
sim.ims.under_relaxation = "simple"
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("simple")
df2 = read_solver_csv()
df2

In [ ]:
plt.plot(df2.total_inner_iterations, df2.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Add cooley under-relaxation

Set `under_relaxation` to `cooler`, write the revised `IMS` file, and run the model 

In [ ]:
sim.ims.under_relaxation = "cooley"
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("cooley")
df3 = read_solver_csv()
df3

In [ ]:
plt.plot(df3.total_inner_iterations, df3.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Add delta-bar-delta under-relaxation

Set `under_relaxation` to `dbd`, write the revised `IMS` file, and run the model 

In [ ]:
sim.ims.under_relaxation = "dbd"
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("dbd")
df4 = read_solver_csv()
df4

In [ ]:
plt.plot(df4.total_inner_iterations, df4.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Add backtracking

Set `under_relaxation` to `none`, set `backtracking_number` to 5, write the revised `IMS` file, and run the model 

In [ ]:
sim.ims.under_relaxation = "none"
sim.ims.backtracking_number = 5
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("backtracking")
df5 = read_solver_csv()
df5

In [ ]:
plt.plot(df5.total_inner_iterations, df5.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Add backtracking and under-relaxation

Set `under_relaxation` to `simple`, set `backtracking_number` to 5, write the revised `IMS` file, and run the model 

In [ ]:
sim.ims.under_relaxation = "simple"
sim.ims.backtracking_number = 5
sim.ims.write()

In [ ]:
sim.run_simulation(processors=processors)

### Read and plot the new results

In [ ]:
df_index.append("backtracking+simple")
df6 = read_solver_csv()
df6

In [ ]:
plt.plot(df6.total_inner_iterations, df6.solution_inner_dvmax)
plt.gca().set_yscale("symlog")

## Create a summary table

Evaluate the total number of iterations and maximum head change at the end of the simulation

In [ ]:
df_list = [df, df1, df2, df3, df4, df5, df6]
df_all = pd.concat([d.iloc[-1:] for d in df_list], ignore_index=True)
df_all.drop(
    columns=[
        "totim",
        "kstp",
        "kper",
        "ninner",
        "solution_inner_dvmax_model",
        "solution_inner_rmax_model",
        "solution_inner_alpha",
        "solution_inner_omega",
    ],
    inplace=True,
)
df_all["simulation"] = df_index
df_all.insert(0, "converged", True)
df_all.loc[
    (df_all["nouter"] == 100) | (df_all["nouter"] == 300), "converged"
] = False
df_all.set_index(keys=["simulation"], inplace=True)

In [ ]:
df_all